In [1]:
!pip install fasttext

In [2]:
import fasttext
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/bert_dataset/assignment')
os.getcwd()
os.listdir()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['test.txt',
 'train.txt',
 'myfile.txt',
 'final.txt',
 'ep_10.bin',
 'ep_100.bin',
 'train_final_trans.txt',
 'test_final_trans.txt',
 'new_final.txt']

In [0]:
import string
train_data = []
ans = 0
with open("train.txt", "r", encoding="utf-8") as f:
    tweet = {}
    for row in f.readlines():
        row_data = row.strip().split()
        if len(row_data)==0 or len(row_data)==1:
            continue
        if row_data[0]=="meta":
            train_data.append(tweet)
            tweet = {}
            tweet["id"] = row_data[1]
            tweet["class"] = row_data[2]
            tweet["text"] = []
            tweet["schema"] = []
        else:
            tweet["text"].append(row_data[0])
            tweet["schema"].append(row_data[1])
train_data = train_data[1:]

for i in train_data:
    new_tok=[]
    tokens=i['text']
    tokens=list(filter(lambda a: a !='//' and a!='/' and a!='co' and a!='https' and a!='...' and a!='@', tokens))

    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
    i['sentence']=x

In [4]:
test_data = []
ans = 0
with open("test.txt", "r", encoding="utf-8") as f:
    tweet = {}
    for row in f.readlines():
        row_data = row.strip().split()
        if len(row_data)==0 or len(row_data)==1:
            continue
        if row_data[0]=="meta":
            test_data.append(tweet)
            tweet = {}
            tweet["id"] = row_data[1]
            tweet["class"] = row_data[2]
            tweet["text"] = []
            tweet["schema"] = []
        else:
            tweet["text"].append(row_data[0])
            tweet["schema"].append(row_data[1])
test_data = test_data[1:]

for i in test_data:
    new_tok=[]
    tokens=i['text']
    tokens=list(filter(lambda a: a !='//' and a!='/' and a!='co' and a!='https' and a!='...' and a!='@', tokens))

    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
    i['sentence']=x
print(len(test_data))

1868


In [0]:
import re
for i in test_data:
    token=i['text']
    temp_token=[]
    j=0
    while j<len(token):
        if(token[j]=='@'):
            j=j+2
        elif(token[j]=='https'):
            break
        elif(token[j]=='_' or token[j]=='-' or token[j]=='#' or token[j]=='+'):
            j=j+1
        else:    
            temp_token.append(token[j])
            j=j+1
            #print(j)
    
        
    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in temp_token]).strip()
    i['uncleaned']=x
    k=re.sub(r"\.+","",x)
    i['clean']=k.encode('ascii', 'ignore').decode('ascii')

In [0]:
import re
for i in train_data:
    token=i['text']
    temp_token=[]
    j=0
    while j<len(token):
        if(token[j]=='@'):
            j=j+2
        elif(token[j]=='https'):
            break
        elif(token[j]=='_' or token[j]=='-' or token[j]=='#' or token[j]=='+'):
            j=j+1
        else:    
            temp_token.append(token[j])
            j=j+1
            #print(j)
    
        
    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in temp_token]).strip()
    i['uncleaned']=x
    k=re.sub(r"\.+","",x)
    i['clean']=k.encode('ascii', 'ignore').decode('ascii')

In [7]:
import re
for i in train_data[:10]:
  print(i['clean'].lower())

pakistan ka ghra tauq he pakistan israel ko tasleem nahein kerta isko palestine kehta he occupied palestine
madarchod mulle ye mathura me nahi dikha tha jab mullo ne hindu ko iss liye mara ki vo lasse ki paise mag liye the 
manya pradhan mantri mahoday shriman narendra modi ji pradhanmantri banne par hardik badhai tahe dil 
krishna jcb full trend me chal rahi aa
loksabha me janta sirf modi ko vote de rahi thi na ki kisi mp or bjp ko without m 
bhosdike tum pechvade ki tatti hi rahoge bc
love u bhaijan  father son  bharat iambharat bharatthiseid best pic from entire promotions  mashallah 
tumhara pass abh deemagh hai nahi islea google ko apna deemagh banaya hua hai har koi tumhari tarh 
nolo weni ankere o gae this weekend 
aimim lage raho mullo tumhre issi quran faad gyan ki kayal hain duniya allah bhi khus 


In [0]:
#result=full_text.encode('ascii', 'ignore').decode('ascii')
file1 = open("new_final.txt",'w',encoding='utf-8')
write_data=[]

for i in train_data:
  write_data.append(i['clean'].lower()+str('\n'))


file1.writelines(write_data) 


In [0]:
import fasttext

model = fasttext.train_unsupervised('new_final.txt','skipgram', minn=2, maxn=5, dim=300,epoch=150, lr=0.1)


In [0]:
model.save_model("ep_150.bin")

In [11]:
import fasttext

model = fasttext.load_model("ep_150.bin")
#model.get_word_vector('the')

In [12]:
f=model.get_output_matrix()
print(f.shape)
p=model.words
print(len(p))


(5979, 300)
5979


In [16]:
embeddings={}
all_words=model.words
out_vec=model.get_word_vector

for i in all_words:
  embeddings[i]=out_vec(i)

print(len(embeddings))

5979


In [0]:
vocab={}
for i in range(len(all_words)):
  vocab[all_words[i]]=i


In [0]:
import numpy as np

num_words = len(embeddings) + 1
EMBED_DIM=300
embedding_matrix = np.zeros((num_words, EMBED_DIM))
for word,i in vocab.items():
    embedding_vector = embeddings.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embeddings_index = None

In [21]:
embedding_matrix.shape
#print(embedding_matrix[1])

(5980, 300)

In [22]:
import tensorflow as tf
from keras.callbacks import TensorBoard, CSVLogger
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Flatten,LSTM,Conv1D,GlobalMaxPool1D,Dropout,Bidirectional,GRU
from keras.layers.embeddings import Embedding
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.utils.vis_utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.models import load_model
from nltk.corpus import stopwords
import operator
import os
import keras

Using TensorFlow backend.


In [0]:
import re
import string
train_data = []
ans = 0
with open("train.txt", "r", encoding="utf-8") as f:
    tweet = {}
    for row in f.readlines():
        row_data = row.strip().split()
        if len(row_data)==0 or len(row_data)==1:
            continue
        if row_data[0]=="meta":
            train_data.append(tweet)
            tweet = {}
            tweet["id"] = row_data[1]
            tweet["class"] = row_data[2]
            tweet["text"] = []
            tweet["schema"] = []
        else:
            tweet["text"].append(row_data[0])
            tweet["schema"].append(row_data[1])
train_data = train_data[1:]

for i in train_data:
    new_tok=[]
    tokens=i['text']
    tokens=list(filter(lambda a: a !='//' and a!='/' and a!='co' and a!='https' and a!='...' and a!='@', tokens))

    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in tokens]).strip()
    i['sentence']=x
    
for i in train_data:
    token=i['text']
    temp_token=[]
    j=0
    while j<len(token):
        if(token[j]=='@'):
            j=j+2
        elif(token[j]=='https'):
            break
        elif(token[j]=='_' or token[j]=='-' or token[j]=='#' or token[j]=='+'):
            j=j+1
        else:    
            temp_token.append(token[j])
            j=j+1
            #print(j)
    
        
    x="".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in temp_token]).strip()
    i['uncleaned']=x
    k=re.sub(r"\.+","",x)
    i['clean']=k.encode('ascii', 'ignore').decode('ascii')

In [0]:
for i in train_data:
  if(i['class']=='negative'):
    i['label']=0
  elif(i['class']=='positive'):
    i['label']=1
  else:
    i['label']=2


In [0]:
def preprocess_statement(statement):
    #statement = [w for w in statement.split(' ') if w not in stopwords.words('english')]
    #statement = ' '.join(statement)
    text = text_to_word_sequence(statement)
    val = [0] * 30
    val = [vocab[t] for t in text if t in vocab] 
    return val

#x_train=np.zeros(shape=(len(train_data),30))
train_lst=[]
for i in train_data:
  train_lst.append(preprocess_statement(i['clean']))



In [0]:
input_sequences = np.array(sequence.pad_sequences(train_lst,   
                          maxlen=30, padding='post'))


y_train=np.zeros(shape=(len(train_data),1))

for i in range(len(train_data)):
  y_train[i]=train_data[i]['label']

y_train = keras.utils.to_categorical(y_train, num_classes=3)

In [93]:
vocab_length = len(vocab.keys())
hidden_size = EMBED_DIM 
lstm_size = 300
num_steps = 30
num_epochs = 30
batch_size = 32



model_lstm = Sequential()
model_lstm.add(Embedding(vocab_length+1, EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=False))
model_lstm.add(Bidirectional(LSTM(hidden_size,dropout=0.2, activation='relu')))
model_lstm.add(Dense(3, activation='softmax'))


#statement_input = Input(shape=(num_steps,), dtype='int32', name='main_input')
#x = Embedding(vocab_length+1,EMBED_DIM,weights=[embedding_matrix],input_length=num_steps,trainable=True)(statement_input) 
#lstm_in = Bidirectional(LSTM(lstm_size,dropout=0.2,return_sequences=True, activation='relu'))(x)
#lstm_in = Bidirectional(LSTM(500,dropout=0.2,activation='relu'))(lstm_in)
#dense=Dense(3, activation='softmax', name='main_output')(lstm_in)

#model_lstm = Model(inputs=[statement_input], outputs=[dense])
model_lstm.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, 30, 300)           1794000   
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 600)               1442400   
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 1803      
Total params: 3,238,203
Trainable params: 1,444,203
Non-trainable params: 1,794,000
_________________________________________________________________


In [94]:
name='bhai'
sgd = optimizers.SGD(lr=0.025, clipvalue=0.3, nesterov=True)
adam = optimizers.Adam(lr=0.0075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model_lstm.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
#tb = TensorBoard()
#csv_logger = keras.callbacks.CSVLogger('training_lstm_6.log')
filepath= name+"_weights_best.hdf5"
checkpoint = keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy',verbose=1, save_best_only=True, mode='max')
    
model_lstm.fit(input_sequences, y_train,
          batch_size=32,
          epochs=30,
          verbose=1,
          validation_data=(input_sequences_test, y_test))

Train on 15130 samples, validate on 1868 samples
Epoch 1/30
15130/15130 [==============================] - 79s 5ms/step - loss: 0.9622 - acc: 0.5694 - val_loss: 0.9565 - val_acc: 0.5589
Epoch 2/30
  800/15130 [>.............................] - ETA: 1:00 - loss: 1.1157 - acc: 0.6138

KeyboardInterrupt: ignored

In [0]:
for i in test_data:
  if(i['class']=='negative'):
    i['label']=0
  elif(i['class']=='positive'):
    i['label']=1
  else:
    i['label']=2

test_lst=[]
for i in test_data:
  test_lst.append(preprocess_statement(i['clean']))

input_sequences_test = np.array(sequence.pad_sequences(test_lst,   
                          maxlen=30, padding='post'))


y_test_n=np.zeros(shape=(len(test_data),1))

for i in range(len(test_data)):
  y_test_n[i]=test_data[i]['label']

y_test = keras.utils.to_categorical(y_test_n, num_classes=3)

In [0]:
score = model_lstm.evaluate(input_sequences_test, y_test, verbose=0)
score
pred=model_lstm.predict(input_sequences_test)

In [0]:
predc=np.argmax(pred, axis=1)


In [88]:
from sklearn.metrics import classification_report

print(classification_report(y_test_n,predc))

              precision    recall  f1-score   support

         0.0       0.61      0.61      0.61       532
         1.0       0.62      0.56      0.59       582
         2.0       0.55      0.59      0.57       754

    accuracy                           0.59      1868
   macro avg       0.59      0.59      0.59      1868
weighted avg       0.59      0.59      0.59      1868

